In [ ]:
#1

import os
import pandas as pd
import re
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors

import numpy as np
import collections
import pickle

os.chdir("/Users/aklasnja/2021/")

#ds = 'gov2'
#f_topics = 'reference_tables/topics.gov2.701-850.bm25.map.dataset.csv'

#ds = 'cw12b13'
#f_topics = 'reference_tables/topics.clueweb12b13.201-300.bm25.map.dataset.csv'

#ds = 'cw09b'
#f_topics = 'reference_tables/topics.clueweb09b.1-200.bm25.map.dataset.csv'

#ds = 'robust04'
#f_topics = 'reference_tables/topics.robust04.bm25.map.dataset.csv'

ds = 'msmarco'
f_topics = 'reference_tables/topics.msmarco.bm25.map.dataset.csv'

f_bm25 = os.path.join('retrievals',ds,ds+'_bm25')
f_qrels = os.path.join('retrievals',ds,'qrels.msmarco-passage.dev-subset.txt')
f_liwc_bm25 = os.path.join('retrievals',ds,ds+'_liwc_retrieved.csv')
f_liwc_qrels = os.path.join('retrievals',ds,ds+'_liwc_qrels.csv')

#bias calc
at_ranklist = [5, 10, 20, 30, 50]
save_pkl = {'tc': os.path.join('retrievals','bias',ds+'_tc.pkl'), \
            'bool': os.path.join('retrievals','bias',ds+'_bool.pkl'), \
            'tf': os.path.join('retrievals','bias',ds+'_tf.pkl')}
save_qpkl = {'tc': os.path.join('retrievals','bias',ds+'_qrels_tc.pkl'), \
            'bool': os.path.join('retrievals','bias',ds+'_qrels_bool.pkl'), \
            'tf': os.path.join('retrievals','bias',ds+'_qrels_tf.pkl')}

wordlist = 'reference_tables/wordlist_genderspecific.txt'

missing_docs = os.path.join('retrievals',ds,ds+'_missing_docs')

f_bias = [os.path.join('retrievals',ds,ds+'_tc_bm25.csv'),
          os.path.join('retrievals',ds,ds+'_tf_bm25.csv'),
          os.path.join('retrievals',ds,ds+'_bool_bm25.csv')]
f_qbias = [os.path.join('retrievals',ds,ds+'_tc_qrels.csv'),
          os.path.join('retrievals',ds,ds+'_tf_qrels.csv'),
          os.path.join('retrievals',ds,ds+'_bool_qrels.csv')]

#queries_gender_annotated_path = "reference_tables/queries_gender_annotated.txt"
#qryids_filter = set(qryids_filter)

#a reference for model classes
ref_models = pd.read_csv('reference_tables/ref_models.csv')

In [ ]:
#2

d = pd.read_csv(f_topics)
d = d.set_index('qid')
d.rename(columns={'abstractqueryexpansion': 'query.0'}, inplace=True)

#original layout - separate queries., method names
q_cols = d[[col for col in d.columns if 'query.' in col]]
m_cols = d[[col for col in d.columns if 'method.' in col]]

qids = q_cols.index.tolist()

In [ ]:
f_ref = ds + '_ref.csv'
ref = pd.read_csv(os.path.join('retrievals',ds,f_ref))

In [ ]:
#3

#a reference for query labels

ref = pd.DataFrame(columns = ['qid','qid_text','query_no','query_text','method'])
for i, qid in enumerate(qids):
    no_queries = q_cols.loc[qid,].count()
    for j in range(0, no_queries): 
        if j > 0:
            ref = ref.append({'qid':qid, \
                              'qid_text':q_cols.loc[qid, 'query.0'], \
                              'query_no':'query.'+str(j), \
                              'query_text':q_cols.loc[qid, 'query.'+str(j)], \
                              'method':m_cols.loc[qid, 'method.'+str(j)]}, ignore_index=True)
        else:
            ref = ref.append({'qid':qid, \
                              'qid_text':q_cols.loc[qid, 'query.0'], \
                              'query_no':'query.'+str(j), \
                              'query_text':q_cols.loc[qid, 'query.'+str(j)], \
                              'method':'none'}, ignore_index=True)

ref = ref.set_index(['method'])
ref = ref.join(ref_models.set_index(['method']), how='left')

f_ref = ds + '_ref.csv'
ref.to_csv(os.path.join('retrievals',ds,f_ref), index=True, header=True, sep=',')

#ref = ref.applymap(lambda x: x.strip() if type(x) == str else x)
#ref = ref.query('qid_text != query_text | method == "none"')

In [ ]:
#4

#retrieved docid lists - not sure if all documents are in the folder and not empty though
r = pd.read_csv(f_bm25, delimiter='\s+', names=['qid', 'query_no', 'method', 'rank', 'docid'])
r = r.query('rank <= 50')
r = ref.reset_index(drop=False).set_index(['qid', 'query_no', 'method']).join(r.set_index(['qid', 'query_no', 'method']), how='left')  
r = r.reset_index(drop=False)

#qrels lists
qr = pd.read_csv(f_qrels, delimiter='\s+', names=['qid', 'rank', 'docid', 'fill'])
qr = qr.set_index(['qid']).join(ref.query("method == 'none'").reset_index(drop=False).set_index(['qid']), how='left')
qr = qr.reset_index(drop=False)

def iter_(r, r0): 
    xr = r.groupby(r.index).mean()  
    xr0 = r0.groupby(r0.index).mean() 
    xr0 = xr0[xr.columns]   
    for row in xr.iloc[0:1].iterrows():
        tmp = row[1] - xr0.loc[row[0][0]]
        ratio = tmp.to_frame().transpose()
        ratio['qid'] = row[0][0]
        ratio['method'] = row[0][1]
        ratio = ratio.set_index(['qid', 'method'])       
    for row in xr.iloc[1:].iterrows():
        ratio.loc[(row[0]),:] = row[1] - xr0.loc[row[0][0]]    
    return(ratio)
#r.to_csv(os.path.join('retrievals',ds,'ck_r.csv'), index=True, header=True, sep=',')

In [ ]:
#5

#liwc variables for all documents

l = pd.read_csv(f_liwc_bm25, delimiter=',', header='infer', engine='python')
if (ds=='msmarco'):
    l['docid'] = l['A'] #manually delete col containing text out of LIWC results
    l = l.set_index(['docid']).drop(columns=['A'])
else:
    l['docid'] = l['Filename'].str.replace('.txt', '', regex=False)
    l = l.set_index(['docid']).drop(columns=['Filename','Segment'])
d = pd.read_csv(f_liwc_qrels, delimiter=',', header='infer', engine='python')
if (ds=='msmarco'):
    d['docid'] = d['A']
    d = d.set_index(['docid']).drop(columns=['A'])
else: 
    d['docid'] = d['Filename'].str.replace('.txt', '', regex=False)
    d = d.set_index(['docid']).drop(columns=['Filename','Segment'])

sel = ['female', 'male',
       'affect', 'posemo', 'negemo', 'anx', 'anger', 'sad', \
       'social', 'family', 'friend', \
       'cogproc', 'insight', 'cause', 'discrep', 'tentat', 'certain', 'differ', \
       'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual', 'ingest', \
       'drives', 'affiliation', 'achieve', 'power', 'reward', 'risk', \
       'focuspast', 'focuspresent', 'focusfuture', \
       'relativ', 'motion', 'space', 'time', 'work', 'leisure', 'home', 'money', 'relig', 'death'] 
#with open(os.path.join('retrievals',ds,ds+'_liwc_retrieved.pkl'), 'wb') as f:
#    pickle.dump(l, f)
    
#with open(os.path.join('retrievals',ds,ds+'_liwc_qrels.pkl'), 'wb') as f:
#    pickle.dump(d, f)
    
#with open(os.path.join('retrievals',ds,ds+'_liwc_retrieved.pkl'), 'rb') as f:
    #l = pickle.load(f)

In [ ]:
#7

#join to liwc results

r = r.dropna().set_index(['docid'], drop=True)
r = r.join(l[sel], how='left')
r[sel] = r[sel].div(100)
r = r.reset_index(drop=True)#----------rowid index

qr = qr.reset_index(drop=False).set_index(['docid']).drop(columns=['rank', 'fill'])
qr = qr.join(d[sel], how='left')
qr = qr.set_index(['qid'], drop=True)
qr[sel] = qr[sel].div(100) #----------qid index

with open(os.path.join('retrievals',ds,ds+'_r_l.pkl'), 'wb') as f:
    pickle.dump(r, f)
qr.to_csv(os.path.join('retrievals',ds,'qr.csv'), index=True, header=True, sep=',')

#r.isnull().sum()
#missing_docs = r[~r.isna().any(axis=1)]['docid']
#missing_docs = pd.DataFrame(r[r.isna().any(axis=1)]['docid'].unique())
#missing_docs.to_csv(os.path.join('retrievals',ds,'missing_docs.txt'), index=False, header=False, sep=',')
#qr[qr.isna().any(axis=1)]

In [ ]:
with open(os.path.join('retrievals',ds,ds+'_r_l.pkl'), 'rb') as f:
    r = pickle.load(f)

In [ ]:
#8

#relative: QE query vector - original query vector

#reduce qids to those w/ more than original query
exclude_qids = q_cols[q_cols['query.1'].isna()].index.tolist()

#set r, r0
ra = r[~r['qid'].isin(exclude_qids)] 
r0a = ra.query('method == "none"')

ra.to_csv(os.path.join('retrievals',ds,'r_liwc_qe.csv'), index=True, header=True, sep=',')

ratio_df = list()
for n in at_ranklist:
    r0_df = ra.query('rank <= @n').set_index(['qid','method']).filter(sel) #----------qid method index
    r0a_df = r0a.query('rank <= @n').set_index(['qid']).filter(sel) #----------qid index
    ratio = iter_(r0_df, r0a_df)
    ratio.columns = ratio.columns + '_' + str(n)
    ratio_df.append(ratio)
    
ratioa = reduce(lambda df1, df2: pd.merge(df1, df2, on=['qid','method']), ratio_df)
ratioa.to_csv(os.path.join('retrievals',ds,'ratio_liwc_qe.csv'), index=True, header=True, sep=',')

In [ ]:
#9

#relative: QE query vector - qrel vector

#reduce qids to those w/ more than original query
exclude_qids = q_cols[q_cols['query.1'].isna()].index.tolist()

#reduce qids to only those w/ qrels
exclude_qids = exclude_qids + list(set(qids).difference(set(qr.index.values)))

#set r, r0
rb = r.reset_index(drop=False)
rb = rb[~rb['qid'].isin(exclude_qids)]

r0b = qr.reset_index(drop=False) 
r0b = r0b[~r0b['qid'].isin(exclude_qids)].set_index(['qid'], drop=True).filter(sel) #----------qid index
#r0b.to_csv(os.path.join('retrievals',ds,'r_liwc_qr.csv'), index=True, header=True, sep=',')

ratio_df = list()
for n in at_ranklist:
    rb_df = rb.query('rank <= @n').set_index(['qid','method']).filter(sel) #----------qid method index
    ratio = iter_(rb_df, r0b)
    ratio.columns = ratio.columns + '_' + str(n)
    ratio_df.append(ratio)

ratiob = reduce(lambda df1, df2: pd.merge(df1, df2, on=['qid','method']), ratio_df)
ratiob.to_csv(os.path.join('retrievals',ds,'ratio_liwc_qr.csv'), index=True, header=True, sep=',')

In [ ]:
#10

def calc_run_bias(experiments, docs_bias, save_pkl):   
    runs_bias = {}
    for exp_name in experiments:

        run_path = experiments[exp_name]
        runs_bias[exp_name] = {}

        for method in save_pkl:
            runs_bias[exp_name][method] = {}

        #open ranked search results
        with open(run_path) as fr:
            qryid_cur = 0

            for i, line in enumerate(fr):
                #if (i % 5000000 == 0) and (i != 0):
                    #print ('line', i)

                vals = line.strip().split(' ')
                if len(vals) == 5:
                    qryid = vals[0]+'-'+vals[2]
                    docid = int(vals[4])

                    if qryid != qryid_cur:
                        for method in save_pkl:
                            runs_bias[exp_name][method][qryid] = []
                        qryid_cur = qryid

                    for method in save_pkl:
                        try:
                            runs_bias[exp_name][method][qryid].append(docs_bias[method][docid])   
                        except:
                            print(docid)
                            runs_bias[exp_name][method][qryid].append(None) 
                            with open(missing_docs, 'a') as miss:
                                miss.write(f'{docid}\n')
                            continue

        for method in save_pkl:
            print (method, len(runs_bias[exp_name][method].keys()))
    return runs_bias

def calc_RaB_q(bias_list, at_rank):
    bias_val = np.mean([x[0] for x in bias_list[:at_rank]])
    bias_feml_val = np.mean([x[1] for x in bias_list[:at_rank]])
    bias_male_val = np.mean([x[2] for x in bias_list[:at_rank]])
    
    return bias_val, bias_feml_val, bias_male_val
         
def calc_ARaB_q(bias_list, at_rank):
    vals = []
    feml_vals = []
    male_vals = []
    for t in range(at_rank):
        if len(bias_list) >= t+1:
            val_RaB, feml_val_RaB, male_val_RaB = calc_RaB_q(bias_list, t+1)
            vals.append(val_RaB)
            feml_vals.append(feml_val_RaB)
            male_vals.append(male_val_RaB)
            
    bias_val = np.mean(vals)
    bias_feml_val = np.mean(feml_vals)
    bias_male_val = np.mean(male_vals)
    
    return bias_val, bias_feml_val, bias_male_val

def calc_qry_bias(experiments, docs_bias, runs_bias, at_ranklist, ref):
    qry_bias_RaB = {}
    qry_bias_ARaB = {}
    for exp_name in experiments:
        qry_bias_RaB[exp_name] = {}
        qry_bias_ARaB[exp_name] = {}

        for method in docs_bias:
            qry_bias_RaB[exp_name][method] = {}
            qry_bias_ARaB[exp_name][method] = {}

            for at_rank in at_ranklist:
                qry_bias_RaB[exp_name][method][at_rank] = {}
                qry_bias_ARaB[exp_name][method][at_rank] = {}

                for qry_id in runs_bias[exp_name][method]:
                    qry_bias_RaB[exp_name][method][at_rank][qry_id] = calc_RaB_q(runs_bias[exp_name][method][qry_id], at_rank)
                    qry_bias_ARaB[exp_name][method][at_rank][qry_id] = calc_ARaB_q(runs_bias[exp_name][method][qry_id], at_rank)

    experiment = list(experiments.keys())
    bias = list(docs_bias.keys())
    print(experiment)
    print(bias)

    bias_df = list()
    for i in experiment:
        for j in bias:
            df = pd.DataFrame(qry_bias_RaB[i][j], columns=list(qry_bias_RaB[i][j].keys())).reset_index()
            df[['qid', 'method']] = df['index'].str.split('-', 1, expand=True)
            df = df.drop(columns=['index']).set_index(['qid', 'method'])
            for l in df.columns:
                df['bias_'+str(l)], df['feml_'+str(l)], df['male_'+str(l)] = df[l].str
                df = df.drop(columns=[l])
            df.columns = j + '_RaB_' + df.columns
            out = df
            df = pd.DataFrame(qry_bias_ARaB[i][j], columns=list(qry_bias_ARaB[i][j].keys())).reset_index()
            df[['qid', 'method']] = df['index'].str.split('-', 1, expand=True)
            df = df.drop(columns=['index']).set_index(['qid', 'method'])
            for l in df.columns:
                df['bias_'+str(l)], df['feml_'+str(l)], df['male_'+str(l)] = df[l].str
                df = df.drop(columns=[l])
            df.columns = j + '_ARaB_' + df.columns
            bias_df.append(out.join(df, how='outer'))
        
    #bias output
    b = reduce(lambda df1, df2: pd.merge(df1, df2, on=['qid','method']), bias_df)
    ref = ref.reset_index(drop=False).set_index(['qid','method'], drop=True)
    b.index = b.index.set_levels(b.index.levels[0].astype(int), level=0)
    b = b.join(ref, how='left')
    return b



In [ ]:
#11

# runs bias 

docs_bias = {}
for method in save_pkl:
    print (method)
    with open(save_pkl[method], 'rb') as fr:
        docs_bias[method] = pickle.load(fr)

experiments = {'bm25': f_bm25}
runs_bias = calc_run_bias(experiments, docs_bias, save_pkl)

b = calc_qry_bias(experiments, docs_bias, runs_bias, at_ranklist, ref)
b.to_csv(os.path.join('retrievals',ds,ds+'_bias_retrieved.csv'), index=True, header=True, sep=',')

In [ ]:
#12

#relative: QE query vector - original query vector

#reduce qids to those w/ more than original query
exclude_qids = q_cols[q_cols['query.1'].isna()].index.tolist()

b = b.reset_index(drop=False)
ba = b[~b['qid'].isin(exclude_qids)] 

ba.to_csv(os.path.join('retrievals',ds,'r_bias_qe.csv'), index=True, header=True, sep=',')

#set r, r0
b0a = ba.query('method == "none"').set_index(['qid'], drop=True).select_dtypes(['number']) #----------qid index
ba = ba.set_index(['qid','method'], drop=True).select_dtypes(['number']) #----------qid method index

ratioa = iter_(ba, b0a)
ratioa.to_csv(os.path.join('retrievals',ds,'ratio_bias_qe.csv'), index=True, header=True, sep=',')

In [ ]:
#13

def calc_RaB_q_qrels(bias_list):
    bias_val = np.mean([x[0] for x in bias_list])
    bias_feml_val = np.mean([x[1] for x in bias_list])
    bias_male_val = np.mean([x[2] for x in bias_list])
    
    return bias_val, bias_feml_val, bias_male_val

def calc_run_bias_qrels(experiments, docs_bias, save_pkl):     
    runs_bias = {}
    for exp_name in experiments:

        run_path = experiments[exp_name]
        runs_bias[exp_name] = {}

        for method in save_pkl:
            runs_bias[exp_name][method] = {}

        #open ranked search results
        with open(run_path) as fr:
            qryid_cur = 0

            for i, line in enumerate(fr):
                if (i % 5000000 == 0) and (i != 0):
                    print ('line', i)

                vals = line.strip().split(' ')
                if len(vals) == 4:
                    qryid = vals[0]
                    docid = vals[2]

                    if qryid != qryid_cur:
                        for method in save_pkl:
                            runs_bias[exp_name][method][qryid] = []
                        qryid_cur = qryid

                    for method in save_pkl:
                        runs_bias[exp_name][method][qryid].append(docs_bias[method][docid])    

        for method in save_pkl:
            print (method, len(runs_bias[exp_name][method].keys()))
    return runs_bias

def calc_qry_bias_qrels(experiments, docs_bias, runs_bias, ref):
    qry_bias_RaB = {}

    for exp_name in experiments:
        qry_bias_RaB[exp_name] = {}

        for method in docs_bias:
            qry_bias_RaB[exp_name][method] = {}

            for qry_id in runs_bias[exp_name][method]:
                qry_bias_RaB[exp_name][method][qry_id] = calc_RaB_q_qrels(runs_bias[exp_name][method][qry_id])

    experiment = list(experiments.keys())
    bias = list(docs_bias.keys())
    print(experiment)
    print(bias)
    
    bias_df = list()
    for i in experiment:
        for j in bias:
            df = pd.DataFrame(qry_bias_RaB[i][j], columns=list(qry_bias_RaB[i][j].keys())).reset_index()
            df = df.T
            df = df.rename_axis("qid").drop(index='index')
            df.columns = [j+'_RaB_bias', j+'_RaB_feml', j+'_RaB_male'] 
            bias_df.append(df)
        
    #bias output
    b = reduce(lambda df1, df2: pd.merge(df1, df2, on=['qid']), bias_df)
    ref = ref.reset_index(drop=False).set_index(['qid'], drop=True).query('method == "none"')
    b.index = b.index.astype(int)
    b = b.join(ref, how='left')
    return b


In [ ]:
#14   

#saved document bias values
docs_bias = {}
for method in save_qpkl:
    print (method)
    with open(save_qpkl[method], 'rb') as fr:
        docs_bias[method] = pickle.load(fr)

experiments = {'qrels': f_qrels}
runs_bias = calc_run_bias_qrels(experiments, docs_bias, save_qpkl)

b0 = calc_qry_bias_qrels(experiments, docs_bias, runs_bias, ref)
b0.to_csv(os.path.join('retrievals',ds,ds+'_bias_qrels.csv'), index=True, header=True, sep=',')

In [ ]:
#14

#relative: QE query vector - qrel vector

#reduce qids to those w/ more than original query
exclude_qids = q_cols[q_cols['query.1'].isna()].index.tolist()

#reduce qids to only those w/ qrels
exclude_qids = exclude_qids + list(set(qids).difference(set(qr.index.values)))

#set r, r0
bb = b[~b['qid'].isin(exclude_qids)] 
bb = bb.set_index(['qid','method']).select_dtypes(['number']).filter(like='_RaB',axis=1) #----------qid method index

bb.to_csv(os.path.join('retrievals',ds,'r_bias_qr.csv'), index=True, header=True, sep=',')

b0_df = list()
for i in at_ranklist:   
    df = b0.select_dtypes(['number'])
    df.columns = df.columns + '_' + str(i)
    b0_df.append(df)
b0b = reduce(lambda df1, df2: pd.merge(df1, df2, on=['qid']), b0_df) #----------qid index
b0b = b0b[bb.columns]

ratiob = iter_(bb, b0b)
ratiob.to_csv(os.path.join('retrievals',ds,'ratio_bias_qr.csv'), index=True, header=True, sep=',')